In [2]:
!pip install qwen-vl-utils[decord]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 44.4 MB/s eta 0:00:00


In [3]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import json

In [4]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [9]:
def classify_image(image_path):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": "Analyze the image and detect the following:"},
                {"type": "text", "text": "1. Number of people in the image."},
                {"type": "text", "text": "2. Direction they are looking (at-system, left, right, up, down)."},
                {"type": "text", "text": "3. Whether any communication devices are present (true/false)."},
            ],
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")


    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )


    result = {
        "people_count": None,
        "direction-looking": None,
        "communication_device_present": None,
    }

    try:
        raw_response = output_text[0].strip()
        print("Model Output:", raw_response)
        lines = raw_response.split("\n")
        for line in lines:
            line = line.strip().lower()
            if "number of people" in line:
                result["people_count"] = int(line.split(":")[-1].strip())
            elif "direction they are looking" in line:
                result["direction-looking"] = line.split(":")[-1].strip().capitalize()
            elif "communication devices" in line:
                result["communication_device_present"] = "true" in line

    except Exception as e:
        print("Error parsing model output:", str(e))

    return result

In [11]:
if __name__ == "__main__":
    image_path = "/content/another-person.jpg"
    try:
        result = classify_image(image_path)
        print(json.dumps(result, indent=4))
    except Exception as e:
        print("Error during classification:", str(e))

Model Output: 1. Number of people in the image: 2
2. Direction they are looking: Both are looking down.
3. Whether any communication devices are present: False
{
    "people_count": 2,
    "direction-looking": "Both are looking down.",
    "communication_device_present": false
}


In [12]:
if __name__ == "__main__":
    image_path = "/content/looking-up-drinking-water.jpg"
    try:
        result = classify_image(image_path)
        print(json.dumps(result, indent=4))
    except Exception as e:
        print("Error during classification:", str(e))

Model Output: 1. Number of people in the image: 1
2. Direction they are looking: Up
3. Whether any communication devices are present: False
{
    "people_count": 1,
    "direction-looking": "Up",
    "communication_device_present": false
}


In [13]:
if __name__ == "__main__":
    image_path = "/content/looking-down (4).jpg"
    try:
        result = classify_image(image_path)
        print(json.dumps(result, indent=4))
    except Exception as e:
        print("Error during classification:", str(e))

Model Output: 1. Number of people in the image: 1
2. Direction they are looking: The person is looking straight ahead.
3. Whether any communication devices are present: False
{
    "people_count": 1,
    "direction-looking": "The person is looking straight ahead.",
    "communication_device_present": false
}
